This works, with the model from AMAbot.ipynb on 5 Jun 2023 before 11:48am.

It works on a T4 (15GB GPU RAM). But only if you request the high-RAM machines! (otherwise you can't load the base model)

But.. you can't merge LoRA weights back into 8-bit models, and 16-bit model (just loading  without the 8-bit) can't be loaded on a T4.

In [1]:
!nvidia-smi

Mon Jun  5 20:40:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    31W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# # https://jaketae.github.io/study/gpt2/
# #  import locale
# #  locale.getpreferredencoding = lambda: "UTF-8"
!pip install transformers 
!pip install huggingface_hub

!pip install peft
!pip install accelerate
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# from huggingface_hub import notebook_login
# notebook_login()
# raise ValueError

In [4]:
# import transformers
import torch
from transformers import (
            AutoTokenizer, AutoModelForCausalLM,
            DataCollatorForLanguageModeling,
            Trainer, TrainingArguments,
            get_cosine_schedule_with_warmup)
from dataclasses import dataclass
# import fire
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)
from peft import prepare_model_for_int8_training
from transformers import Trainer

import os

device = 'cuda'


from peft import PeftConfig, PeftModel
# config = PeftConfig.from_pretrained(repo_name)


# Loading the base model
tokenizer_inf = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-7B-v0.1", padding_side='left')
base_model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-7B-v0.1", device_map='auto', torch_dtype=torch.float16, load_in_8bit=True)


repo_name= 'AMAbot/AMAbot-7B'

# Loading LoRA  weights
model = PeftModel.from_pretrained(base_model, repo_name) 
model.eval()


model.gradient_checkpointing_disable() # Probably no effect for inference

print(tokenizer_inf.model_max_length)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2048


In [5]:
# prompt = "<human>: What is the sound of one hand clapping?\n<bot>:"
# prompt = "<human>: Q for tszzl: What is the sound of one hand clapping?\n<bot>:"
# prompt = "<human>: Q for FINALLEVEL: What is the sound of one hand clapping?\n<bot>:"
# prompt = "<human>: Q for KarpaT: What is your favorite programming language?\n<bot>:"
# prompt = "<human>: Q for KarpaT: What is your favorite video game?\n<bot>:"
# prompt = "<human>: Q for KarpaT: What are you doing right now?\n<bot>:"
# prompt="<human>: When was the term OG first introduced to the world?\n<bot>:"
# prompt="<human>: Q for FINALLEVEL: When was the term OG first introduced to the world?\n<bot>:"
# prompt = "<human>: Q for tszzl: How is it going?\n<bot>:"
# prompt = "<human>: Q for FINALLEVEL: How is it going?\n<bot>:"
# prompt = "<human>: How is it going?\n<bot>:"
# prompt = "My dream is"
prompts = [

    "<human>: Q for Karpathy: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpathy: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpathy: What is the sound of one hand clapping?\n<bot>:", 
    "<human>: Q for Karpathy: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpathy: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpathy: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpathy: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpathy: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpathy: What are you doing right now?\n<bot>:",

    "<human>: Q for Ice-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Ice-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Ice-T: What is the sound of one hand clapping?\n<bot>:", 
    "<human>: Q for Ice-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Ice-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Ice-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Ice-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Ice-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Ice-T: What are you doing right now?\n<bot>:",

    "<human>: Q for Karpa-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpa-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpa-T: What is the sound of one hand clapping?\n<bot>:", 
    "<human>: Q for Karpa-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpa-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpa-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpa-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpa-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpa-T: What are you doing right now?\n<bot>:",

    ]



tokenizer_inf.pad_token = tokenizer_inf.eos_token 

inputs = tokenizer_inf(prompts, return_tensors='pt',  padding='longest').to(device)

print(inputs.input_ids.shape)
input_length = inputs.input_ids.shape[1]
outputs = model.generate(
    # **inputs, max_new_tokens=64, do_sample=True, temperature=0.7, top_p=0.7, top_k=50, return_dict_in_generate=True
    **inputs, max_new_tokens=64, do_sample=True, top_p=0.7, return_dict_in_generate=True
)
# tokenz = outputs.sequences[:, input_length:]  # If we want to remove the questions from the answers
tokenz = outputs.sequences  
output_list = tokenizer_inf.batch_decode(tokenz)
for x in output_list:
    print(x.replace('<|endoftext|>','')+"\n==" )

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


torch.Size([27, 24])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 55>:55                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:731 in generate                       │
│                                                                                                  │
│    728 │   │   self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generati  │
│    729 │   │   try:                                                                              │
│    730 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  731 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    732 │   │   │   else:                                                                         │
│    733 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    734 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1565 in generate        │
│                                                                                                  │
│   1562 │   │   │   )                                                                             │
│   1563 │   │   │                                                                                 │
│   1564 │   │   │   # 13. run sample                                                              │
│ ❱ 1565 │   │   │   return self.sample(                                                           │
│   1566 │   │   │   │   input_ids,                                                                │
│   1567 │   │   │   │   logits_processor=logits_processor,                                        │
│   1568 │   │   │   │   logits_warper=logits_warper,                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2612 in sample          │
│                                                                                                  │
│   2609 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2610 │   │   │                                                                                 │
│   2611 │   │   │   # forward pass to get next token                                              │
│ ❱ 2612 │   │   │   outputs = self(                                                               │
│   2613 │   │   │   │   **model_inputs,                     

In [1]:
!nvidia-smi

Mon Jun  5 20:44:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# model.merge_and_unload()